In [3]:
import pandas as pd
import matplotlib as mlt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# 1. CSV 로드
df = pd.read_csv('info.csv').drop(columns=['Unnamed: 0']).fillna('')

# 2. 모델 로드
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 3. 텍스트 결합
df['full_text'] = (
    df['name'] + " " +
    df['category'] + " " +
    df['company']  + " " +
    df['first_prize'] + " " +
    df['qual']  

    
)

# 4. 임베딩
print("임베딩 생성 중...")
corpus_embeddings = model.encode(df['full_text'].tolist(), convert_to_tensor=True)

# 5. 사용자 입력
user_input = input("추천받을 카테고리를 입력하세요: ").strip()

# 6. 쿼리 임베딩
query_embedding = model.encode(user_input, convert_to_tensor=True)

# 7. 유사도 계산
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

# 8. 상위 5개
topk = torch.topk(cos_scores, k=5)


top_scores = topk.values.tolist()      # [float, float, ...]
top_indices = topk.indices.tolist()    # [int, int, ...]

print(f"\n▶ 카테고리 '{user_input}' 기반 추천 공모전 TOP 5:")
for score, idx in zip(top_scores, top_indices):
    # idx는 이제 int
    row = df.iloc[idx]
    print(f"  카테고리: {row['category']}")
    print(f"  제목: {row['name']}")
    print(f"  주최/주관: {row['company']}")
    print(f"  참가대상: {row['qual']}")
    print(f"  접수기간: {row['register_start']} ~ {row['register_end']}")
    print(f"  총상금: {row['total_prize']}")
    print(f"  유사도: {score:.4f}\n")


임베딩 생성 중...

▶ 카테고리 '문학' 기반 추천 공모전 TOP 5:
  카테고리: 문학/글/시나리오
  제목: 제2회 법정스님과 함께하는 땅끝해남 행복에세이 공모전
  주최/주관: 도서출판 북산
  참가대상: 제한없음
  접수기간: 2025-04-11 ~ 2025-06-10
  총상금: 1천만원이하
  유사도: 0.5380

  카테고리: 기타
  제목: 제14회 전주 기록물 수집 공모전
  주최/주관: 전주시시
  참가대상: 기타
  접수기간: 2025-03-04 ~ 2025-05-30
  총상금: 다양한 혜택
  유사도: 0.5087

  카테고리: 문학/글/시나리오
  제목: 제 10회 초록우산 감사편지 공모전
  주최/주관: 초록우산
  참가대상: 청소년, 어린이
  접수기간: 2025-03-11 ~ 2025-07-11
  총상금: 다양한 혜택
  유사도: 0.5065

  카테고리: 문학/글/시나리오
  제목: 제26회 도산안창호 글짓기 및 독후감 공모전
  주최/주관: 도산안창호선생기념사업회
  참가대상: 제한없음
  접수기간: 2025-04-01 ~ 2025-09-30
  총상금: 1천만원이하
  유사도: 0.4932

  카테고리: 예체능/미술/음악
  제목: [갤러리비상] 제3회 전국장애인미술공모전
  주최/주관: 고양시장애인종합복지관
  참가대상: 기타
  접수기간: 2025-03-24 ~ 2025-05-08
  총상금: 1천만원이하
  유사도: 0.4754



In [2]:
import torch

# 1) 임베딩 계산 후 저장
torch.save(corpus_embeddings, 'corpus_embeddings.pt')
print("임베딩 텐서를 'corpus_embeddings.pt'로 저장했습니다.")

# 2) 나중에 불러오기
loaded_embeddings = torch.load('corpus_embeddings.pt')
# loaded_embeddings.shape 는 (num_documents, embedding_dim)


임베딩 텐서를 'corpus_embeddings.pt'로 저장했습니다.


C:\Users\KDP-50\AppData\Local\Temp\ipykernel_18980\3011130427.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_embeddings = torch.load('corpus_embeddings.pt')
